In [1]:
%matplotlib inline
import matplotlib.pylab
import pandas as pd
import numpy as np

In [6]:
with open('data/ao_monthly.txt') as f:
    for x in range(5):
        print(next(f))

 1950    1  -0.60310E-01

 1950    2   0.62681E+00

 1950    3  -0.81275E-02

 1950    4   0.55510E+00

 1950    5   0.71577E-01



In [7]:
data = pd.read_fwf('data/ao_monthly.txt',header=None)

In [10]:
data.head(10)

,0,1,2
0,1950,1,-0.060310
1,1950,2,0.626810
2,1950,3,-0.008128
3,1950,4,0.555100
4,1950,5,0.071577
5,1950,6,0.538570
6,1950,7,-0.802480
7,1950,8,-0.851010
8,1950,9,0.357970
9,1950,10,-0.378900


### Look at the options for read_fwf.... what looks relevent?

In [11]:
data = pd.read_fwf('data/ao_monthly.txt',header=None,index_col=0,parse_dates=[[0,1]],infer_datetime_format=True)

In [12]:
data.head()

,2
0_1,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [13]:
data.index

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01',
               ...
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01'],
              dtype='datetime64[ns]', name='0_1', length=798, freq=None)

In [14]:
data.index.names = ['Month']
data.columns = ['Value']
data.head()

,Value
Month,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


### What is the empirical range of dates?

In [15]:
print(min(data.index))
print(max(data.index))

1950-01-01 00:00:00
2016-06-01 00:00:00


### How can we convert to complementary reperesentation?

In [17]:
data.to_period().head()

,Value
Month,
1950-01,-0.060310
1950-02,0.626810
1950-03,-0.008128
1950-04,0.555100
1950-05,0.071577


### More about reading in data
When reading in dates with a pd.read function, you have several time-related parameters you can adjust: parse_dates, infer_datetime, date_parser

</br>

Experiment with these using %timeit to see if there are performance differences 

Hint: infer_datetime_format =True, no date parser provided, what other combos can you come up with?

In [24]:
import timeit

#First, let's see how to use a date_perser:
dateparse = lambda x, y:pd.datetime.strptime('%s-%s'%(x,y),'%Y-%m')
%timeit data = pd.read_fwf('data/ao_monthly.txt',header=None,index_col=0,parse_dates=[[0,1]],date_perser=dateparse)

100 loops, best of 3: 5.55 ms per loop


In [22]:
data.head()

,2
0_1,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [25]:
%timeit data = pd.read_fwf('data/ao_monthly.txt',header=None,index_col=0,parse_dates=[[0,1]],infer_datetime_format=True)

100 loops, best of 3: 5.83 ms per loop


In [26]:
%timeit data = pd.read_fwf('data/ao_monthly.txt',header=None,index_col=0,parse_dates=[[0,1]])

100 loops, best of 3: 5.63 ms per loop


### You can also extract datetimes from existing Data frame columns

In [29]:
# new in pandas 0.18
df = pd.DataFrame({'year':[2015,2016],'month':[2,3],'day':[4,5],'hour':[12,13]})
df

,day,hour,month,year
0,4,12,2,2015
1,5,13,3,2016


In [31]:
pd.to_datetime(df)

0   2015-02-04 12:00:00
1   2016-03-05 13:00:00
dtype: datetime64[ns]

In [34]:
#But
df['value']=17
pd.to_datetime(df)

ValueError: extra keys have been passed to the datetime assemblage: [value]

### Truncating

In [35]:
ts = pd.Series(range(10),index=pd.date_range('7/31/15',freq='M',periods=10))
ts

2015-07-31    0
2015-08-31    1
2015-09-30    2
2015-10-31    3
2015-11-30    4
2015-12-31    5
2016-01-31    6
2016-02-29    7
2016-03-31    8
2016-04-30    9
Freq: M, dtype: int64

In [36]:
#truncating preserves frequency
ts.truncate(before='10/31/2015',after='12/31/2015')

2015-10-31    3
2015-11-30    4
2015-12-31    5
Freq: M, dtype: int64

In [37]:
# you can truncate in a way that does not preserve frequency
ts[[1,6,7]].index

DatetimeIndex(['2015-08-31', '2016-01-31', '2016-02-29'], dtype='datetime64[ns]', freq=None)

In [38]:
# but pandas will try to preserve frequency automatically whenever possible/logical
ts[0:10:2]

2015-07-31    0
2015-09-30    2
2015-11-30    4
2016-01-31    6
2016-03-31    8
Freq: 2M, dtype: int64